<a href="https://colab.research.google.com/github/gitHubAndyLee2020/LLM_Reward_Model_Research/blob/main/Mistral_7B_OpenOrca_Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
! pip install datasets einops sentencepiece tokenizers
! pip install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-7b8e4pjj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-7b8e4pjj
  Resolved https://github.com/huggingface/transformers to commit 897a826d830e8b1e03eb482b165b5d88a7a08d5f
  Installing build dependencies ... don

# Download

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

device = "cuda"
model = AutoModelForCausalLM.from_pretrained(
    "Open-Orca/Mistral-7B-OpenOrca").to(device)
tokenizer = AutoTokenizer.from_pretrained(
    "Open-Orca/Mistral-7B-OpenOrca")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Run

In [ ]:
# Testing random prompt from OpenOrca dataset

%%time
sys_prompt = "You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old."
prompt = "What kind of task would test someone's ability to perform physical reasoning?"

instruct_prefix = "[INST]"
instruct_suffix = "[/INST]"
prefix = "<|im_start|>"
suffix = "<|im_end|>\n"
sys_format = prefix + "system\n" + sys_prompt + suffix
user_format = prefix + "user\n" + prompt + suffix
assistant_format = prefix + "assistant\n"
input_text = sys_format + user_format + assistant_format
input_text = instruct_prefix + input_text + instruct_suffix

generation_config = GenerationConfig(
    max_length=1000, temperature=1.1, top_p=0.95, repetition_penalty=1.0,
    do_sample=True, use_cache=True,
    eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id,
    transformers_version="4.34.0.dev0")

inputs = tokenizer(input_text, return_tensors="pt", return_attention_mask=True).to(device)
outputs = model.generate(**inputs, generation_config=generation_config)

text = tokenizer.batch_decode(outputs)[0]
text += "</s>"
print(text)

<s> [INST]<|im_start|> system
You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.<|im_end|><|im_start|> user
What kind of task would test someone's ability to perform physical reasoning?<|im_end|><|im_start|> assistant
[/INST]

Imagine a task where someone needs to use their body and brain to figure out how to solve a puzzle or accomplish a goal. For example, a task to test someone's ability to perform physical reasoning might be:

A person has to move a big box from one room to another using only their hands and legs. They need to think about the best way to carry it or drag it without hurting themselves or dropping the box. They have to use their brain and body to find a good solution!<|im_end|></s>
CPU times: user 4 s, sys: 8.29 ms, total: 4.01 s
Wall time: 4 s


### Dataset Initialization

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# The dataset identifier is usually in the format "user/dataset_name"
dataset_identifier = "Open-Orca/OpenOrca"
dataset = load_dataset(dataset_identifier)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import pandas as pd

# Convert the "train" split to a DataFrame
df_train = dataset["train"].to_pandas()

# Display the first few rows
df_train.head()

,id,system_prompt,question,response
0,niv.242684,,You will be given a definition of a task first...,"[\n [""AFC Ajax (amateurs)"", ""has ground"", ""Sp..."
1,flan.564327,You are an AI assistant. You will be given a t...,Generate an approximately fifteen-word sentenc...,Midsummer House is a moderately priced Chinese...
2,flan.1875913,"You are a helpful assistant, who always provid...",What happens next in this paragraph?\n\nShe th...,C. She then dips the needle in ink and using t...
3,t0.408370,You are an AI assistant. You will be given a t...,Please answer the following question: I want t...,"Based on the passage, discuss the primary moti..."
4,cot.86217,You are an AI assistant that helps people find...,James runs a TV show and there are 5 main char...,"James pays the minor characters $15,000 each e..."


### Generatic Responses

In [ ]:
def generate_response(sys_prompt, prompt):
  instruct_prefix = "[INST]"
  instruct_suffix = "[/INST]"
  prefix = "<|im_start|>"
  suffix = "<|im_end|>\n"
  sys_format = prefix + "system\n" + sys_prompt + suffix
  user_format = prefix + "user\n" + prompt + suffix
  assistant_format = prefix + "assistant\n"
  input_text = sys_format + user_format + assistant_format
  input_text = instruct_prefix + input_text + instruct_suffix

  generation_config = GenerationConfig(
      max_length=1000, temperature=1.1, top_p=0.95, repetition_penalty=1.0,
      do_sample=True, use_cache=True,
      eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id,
      transformers_version="4.34.0.dev0")

  inputs = tokenizer(input_text, return_tensors="pt", return_attention_mask=True).to(device)
  outputs = model.generate(**inputs, generation_config=generation_config)

  text = tokenizer.batch_decode(outputs)[0]
  text += "</s>"

  return text

In [ ]:
import re

def extract_response(text):
  result = re.search(r'\[/INST\](.*?)</s>', text, re.DOTALL)
  if result:
      extracted_text = result.group(1).strip()
      return extracted_text
  else:
      print("Pattern not found.")
      print("text:")
      print(text)
      return ''

In [ ]:
def get_response(sys_prompt, prompt):
  response = generate_response(sys_prompt, prompt)
  return extract_response(response)

In [ ]:
import os

def generate_answers(df_train, START_IDX, END_IDX):
    # Check if data folder exists. If not, create it.
    if not os.path.exists("data"):
        os.makedirs("data")

    for idx in range(START_IDX, END_IDX):
        sys_prompt = df_train["system_prompt"][idx]
        prompt = df_train["question"][idx]
        answer = get_response(sys_prompt, prompt)
        with open(f"data/{idx}_mistral_7b_openorca.txt", "w", encoding="utf-8") as f:
            f.write(answer)

In [ ]:
START_IDX = 0
END_IDX = 100

In [ ]:
%%time

generate_answers(df_train, START_IDX, END_IDX)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 1051, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 1071, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 1090, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


CPU times: user 12min 32s, sys: 4.06 s, total: 12min 36s
Wall time: 12min 35s


In [ ]:
!zip -r data.zip data/

  adding: data/ (stored 0%)
  adding: data/56_mistral_7b_instruct_v0_1.txt (deflated 42%)
  adding: data/30_mistral_7b_instruct_v0_1.txt (deflated 53%)
  adding: data/28_mistral_7b_instruct_v0_1.txt (deflated 36%)
  adding: data/27_mistral_7b_instruct_v0_1.txt (deflated 47%)
  adding: data/17_mistral_7b_instruct_v0_1.txt (deflated 59%)
  adding: data/54_mistral_7b_instruct_v0_1.txt (deflated 33%)
  adding: data/73_mistral_7b_instruct_v0_1.txt (deflated 55%)
  adding: data/21_mistral_7b_instruct_v0_1.txt (deflated 39%)
  adding: data/19_mistral_7b_instruct_v0_1.txt (stored 0%)
  adding: data/61_mistral_7b_instruct_v0_1.txt (deflated 50%)
  adding: data/7_mistral_7b_instruct_v0_1.txt (deflated 54%)
  adding: data/86_mistral_7b_instruct_v0_1.txt (deflated 55%)
  adding: data/53_mistral_7b_instruct_v0_1.txt (deflated 50%)
  adding: data/62_mistral_7b_instruct_v0_1.txt (deflated 62%)
  adding: data/74_mistral_7b_instruct_v0_1.txt (deflated 47%)
  adding: data/18_mistral_7b_instruct_v0_1.txt

### Debug

In [ ]:
print(tokenizer.eos_token_id)
print(model.config.eos_token_id)
print(tokenizer.pad_token_id)
print(model.config.pad_token_id)

2
2
None
None


In [ ]:
text = tokenizer.decode([1, 2, 32000, 32001])
print(text)

<s></s><|im_end|><|im_start|>


In [ ]:
tokenizer.convert_tokens_to_ids("<|im_end|>")

32000

In [ ]:
# If you get "NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968" error, run this to fix the problem

import locale
locale.getpreferredencoding = lambda: "UTF-8"